## imports

In [1]:
import polars as pl
from polars import col as c
import os, sys, requests
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

True

## constants

In [2]:
OMDB_API_KEY = os.getenv("OMDB_API_KEY")
FILES_DIR = "../data"

## load df

In [4]:
file_name = "movies_df.parquet"
file_path = os.path.join(FILES_DIR, file_name)
movies_df = pl.read_parquet(file_path)

## code

In [6]:
movies_df.head()

index,Title,Year,Viewed,title_eng,title_translation,parsed_date,parsed_title,checked
i64,str,i32,str,str,str,i64,str,str
1,"""Действуй сестра""",null,null,"""Not found""","""Act sister""",null,"""Not found""",null
2,"""Summer lovers""",null,null,"""Summer Lovers_1982_Comedy, Dra…","""Summer lovers""",1982,"""Summer Lovers""","""checked"""
3,"""Джон уик""",null,null,"""John Wick_2014_Action, Crime, …","""John Wick""",2014,"""John Wick""","""checked"""
4,"""Помни""",null,null,"""Remembered_2015_Short, Drama_N…","""Remembered""",2015,"""Remembered""","""checked"""
5,"""Похитители велосипедов""",null,null,"""Not found""","""Cycles of bicycles""",null,"""Not found""",null


In [28]:
len(titles_mapping)

413

In [43]:
titles_mapping

{'Действуй сестра': 'Sister Act',
 'Похитители велосипедов': 'Bicycle Thieves',
 'Академия рашмор': 'Rushmore',
 'Бегущий по лезвию': 'Blade Runner',
 'Мрачные тени': 'Dark Shadows',
 'Агирре, гнев божий': 'Aguirre, the Wrath of God',
 'Роковая восьмерка': 'Hard Eight',
 'На гребне волны': 'Point Break',
 'Братья из гримсби': 'Grimsby',
 'Гадкий койот *': 'Coyote Ugly',
 'Марсианин *': 'The Martian',
 'Касабланка *': 'Casablanca',
 'Boggie nights': 'Boogie Nights',
 '16 свечей': 'Sixteen Candles',
 'Шоу рокки хорора *': 'The Rocky Horror Picture Show',
 'Грязные танцы *': 'Dirty Dancing',
 'Иллюзия обмана': 'Now You See Me',
 'Иррациональный человек *': 'Irrational Man',
 'Jerry maguayer *': 'Jerry Maguire',
 'Области тьмы': 'Limitless',
 'Super mike xxl': 'Magic Mike XXL',
 'Eyes wide closed *': 'Eyes Wide Shut',
 'Золото дураков': "Fool's Gold",
 'Два ствола': '2 Guns',
 'Красавчик *': 'Pretty Boy',
 'Ангел хранитель. *': 'Guardian Angel',
 'Девушка с тату дракона': 'The Girl with th

In [ ]:
# movies_df =
movies_df.with_columns(
    c("Title")
    .replace_strict(
        titles_mapping,
        # replace_strict=False,
    )
    .alias("gpt_parsed_titles")
)

In [41]:
movies_df = movies_df.with_columns(
    c("Title")
    .map_elements(lambda x: titles_mapping.get(x, x))
    .alias("gpt_parsed_titles")
)

/var/folders/yt/5rqq39t16xs0vjb7z99vs0cw0000gn/T/ipykernel_59697/3119976487.py:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  movies_df = movies_df.with_columns(


In [42]:
movies_df.filter(c("checked").is_null())

index,Title,Year,Viewed,title_eng,title_translation,parsed_date,parsed_title,checked,gpt_parsed_titles
i64,str,i32,str,str,str,i64,str,str,str
1,"""Действуй сестра""",null,null,"""Not found""","""Act sister""",null,"""Not found""",null,"""Sister Act"""
5,"""Похитители велосипедов""",null,null,"""Not found""","""Cycles of bicycles""",null,"""Not found""",null,"""Bicycle Thieves"""
6,"""Академия рашмор""",null,null,"""Not found""","""Rashmore Academy""",null,"""Not found""",null,"""Rushmore"""
7,"""Бегущий по лезвию""",null,null,"""Not found""","""Running along the blade""",null,"""Not found""",null,"""Blade Runner"""
13,"""Мрачные тени""",null,null,"""Not found""","""Gloomy shadows""",null,"""Not found""",null,"""Dark Shadows"""
…,…,…,…,…,…,…,…,…,…
617,"""Only Yesterday""",1991,"""12.05.2025""","""Not found""","""Only Yesterday""",null,"""Not found""",null,"""Only Yesterday"""
618,"""Princess Bride""",1987,"""12.06.2025""","""Not found""","""Princess Bride""",null,"""Not found""",null,"""The Princess Bride"""
619,"""Grave of the Fireflies""",1988,"""10.07.2025""","""Not found""","""Grave of the Fireflies""",null,"""Not found""",null,"""Grave of the Fireflies"""


In [29]:
# (movies_df.filter(c("checked").is_null())["Title"].to_list()[47 + 93 + 31 :])

In [ ]:
def fetch_english_title(title):
    try:
        url = f"http://www.omdbapi.com/?apikey={OMDB_API_KEY}&t={requests.utils.quote(title)}"
        response = requests.get(url)
        data = response.json()
        if data.get("Response") == "True" and "Title" in data:
            return (
                data["Title"],
                data["Year"],
                data["Genre"],
                data["Director"],
                data["Country"],
            )
        else:
            print(f"Not found in OMDb: {title}")
            return "Not found"
    except Exception as e:
        print(f"OMDb error for {title}: {e}")
        return "Not found"


# movies = movies.with_columns(
#     title_eng=pl.col("Title").map_elements(fetch_english_title)
# )

In [ ]:
url = (
    f"http://www.omdbapi.com/?apikey={OMDB_API_KEY}&t={requests.utils.quote("Jumanji")}"
)
response = requests.get(url)
data = response.json()
if data.get("Response") == "True" and "Title" in data:
    print(data["Title"])

In [ ]:
# data

{
    "Title": "Jumanji",
    "Year": "1995",
    "Rated": "PG",
    "Released": "15 Dec 1995",
    "Runtime": "104 min",
    "Genre": "Adventure, Comedy, Family",
    "Director": "Joe Johnston",
    "Writer": "Jonathan Hensleigh, Greg Taylor, Jim Strain",
    "Actors": "Robin Williams, Kirsten Dunst, Bonnie Hunt",
    "Plot": "A magic board game summons jungle perils and a missing boy.",
    "Language": "English, French",
    "Country": "United States",
    "Awards": "4 wins & 11 nominations total",
    "Poster": "https://m.media-amazon.com/images/M/MV5BYTFkMjFmODgtYzRiZi00NmQwLTliZWMtMzRhMWQ5ZmY3ZDExXkEyXkFqcGc@._V1_SX300.jpg",
    "Ratings": [
        {"Source": "Internet Movie Database", "Value": "7.1/10"},
        {"Source": "Rotten Tomatoes", "Value": "52%"},
        {"Source": "Metacritic", "Value": "39/100"},
    ],
    "Metascore": "39",
    "imdbRating": "7.1",
    "imdbVotes": "399,171",
    "imdbID": "tt0113497",
    "Type": "movie",
    "DVD": "N/A",
    "BoxOffice": "$100,499,940",
    "Production": "N/A",
    "Website": "N/A",
    "Response": "True",
}

In [ ]:
import pandas as pd
import requests
from deep_translator import GoogleTranslator

# Load the CSV file
df = pd.read_csv("movies.csv")


# Step 1: Translate titles
def translate_title(title):
    try:
        return GoogleTranslator(source="auto", target="en").translate(title)
    except Exception as e:
        print(f"Translation error for {title}: {e}")
        return title  # fallback to original if translation fails


df["title_eng"] = df["Title"].apply(translate_title)

# Step 2: Use OMDb API to fill missing years
OMDB_API_KEY = "your_api_key_here"  # replace with your actual OMDb API key


def fetch_year_from_omdb(title):
    try:
        url = f"http://www.omdbapi.com/?apikey={OMDB_API_KEY}&t={requests.utils.quote(title)}"
        response = requests.get(url)
        data = response.json()
        if data.get("Response") == "True":
            return int(data["Year"]) if "Year" in data else None
        else:
            print(f"Not found in OMDb: {title}")
            return None
    except Exception as e:
        print(f"OMDb error for {title}: {e}")
        return None


df["Year"] = df.apply(
    lambda row: (
        fetch_year_from_omdb(row["title_eng"]) if pd.isna(row["Year"]) else row["Year"]
    ),
    axis=1,
)

# Save the updated CSV
df.to_csv("movies_updated.csv", index=False)
print("✅ Updated CSV saved as 'movies_updated.csv'")

## save df

In [44]:
file_name = "movies_df.parquet"
file_path = os.path.join(FILES_DIR, file_name)
movies_df.write_parquet(file_path)